In [3]:
import re
import os
import unicodedata
import random  # Karıştırma işlemi için eklendi

def turkish_specific_lower(text):
    """
    Türkçeye özgü küçük harf dönüşümü yapar.
    'İ' -> 'i', 'I' -> 'ı'
    """
    text = text.replace('İ', 'i')
    text = text.replace('I', 'ı')
    return text.lower()


def clean_text_revised(text, lang='en'):
    """
    Belirtilen dile ve revize edilmiş gereksinimlere göre tek bir cümleyi temizler.
    """
    # 1. Unicode Normalizasyonu (NFC formu yaygındır)
    text = unicodedata.normalize('NFC', text)

    # 2. Küçük Harfe Dönüştürme (Türkçeye özgü)
    if lang == 'tr':
        text = turkish_specific_lower(text)
    else:
        text = text.lower()

    # 3. (Önerilen) HTML ve URL'leri öncelikle kaldırın
    text = re.sub(r'<[^>]+>', '', text)  # HTML etiketlerini kaldır
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # URL'leri kaldır

    # 4. Belirli noktalama işaretlerini (ve sıralarını) tek bir boşlukla değiştirin
    text = re.sub(r'[.,!?]+', ' ', text)
    # İsteğe bağlı: text = re.sub(r'[.,!?;:]+', ' ', text)

    # 5. Karakter beyaz listesi: Sadece belirtilen karakterler kalsın, diğerlerini kaldır.
    if lang == 'tr':
        text = re.sub(r'[^a-zçğıöşü0-9\s\'_-]', '', text)
    else:  # İngilizce varsayılıyor
        text = re.sub(r'[^a-z0-9\s\'_-]', '', text)

    # 6. Boşluk normalizasyonu: birden çok boşluğu tek boşluğa indir, baştaki/sondaki boşlukları kırp
    text = re.sub(r'\s+', ' ', text).strip()

    return text


def main():
    # Google Drive'a bağlan
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    DRIVE_BASE_PATH = '/content/drive/My Drive/transformer/'
    input_file = os.path.join(DRIVE_BASE_PATH, 'en2tr.txt')
    train_output_file = os.path.join(DRIVE_BASE_PATH, 'en2tr_train.txt')
    valid_output_file = os.path.join(DRIVE_BASE_PATH, 'en2tr_valid.txt')
    split_ratio = 0.9  # %90 eğitim, %10 doğrulama

    cleaned_pairs = []

    try:
        print(f"'{input_file}' dosyası okunuyor ve temizleniyor...")
        with open(input_file, 'r', encoding='utf-8') as infile:
            for line_num, line in enumerate(infile):
                parts = line.strip().split('\t')
                if len(parts) == 2:
                    en_sentence, tr_sentence = parts[0], parts[1]

                    cleaned_en = clean_text_revised(en_sentence, lang='en')
                    cleaned_tr = clean_text_revised(tr_sentence, lang='tr')

                    # Temizlik sonrası boş kalan cümle çiftlerini filtrele
                    if cleaned_en and cleaned_tr:
                        cleaned_pairs.append((cleaned_en, cleaned_tr))
                else:
                    print(f"Uyarı: {line_num + 1}. satırda format hatası: '{line.strip()}' (atlandı)")

        if not cleaned_pairs:
            print("Temizlenecek uygun cümle çifti bulunamadı. İşlem sonlandırılıyor.")
            return

        print(f"Toplam {len(cleaned_pairs)} adet temizlenmiş cümle çifti bulundu.")

        # 1. Veriyi Karıştırma
        print("Cümle çiftleri karıştırılıyor...")
        random.seed(42)  # Tekrarlanabilir karıştırma için (isteğe bağlı, farklı bir sayı da olabilir)
        random.shuffle(cleaned_pairs)

        # 2. Bölme Noktasını Hesaplama
        num_total_pairs = len(cleaned_pairs)
        num_train_pairs = int(num_total_pairs * split_ratio)

        # 3. Veriyi Bölme
        train_data = cleaned_pairs[:num_train_pairs]
        valid_data = cleaned_pairs[num_train_pairs:]

        print(f"Veri bölündü: {len(train_data)} cümle çifti eğitim için, {len(valid_data)} cümle çifti doğrulama için.")

        # 4. Eğitim Verisini Dosyaya Yazma
        with open(train_output_file, 'w', encoding='utf-8') as outfile_train:
            for en_sent, tr_sent in train_data:
                outfile_train.write(f"{en_sent}\t{tr_sent}\n")
        print(f"Eğitim verisi '{train_output_file}' dosyasına yazıldı.")

        # 5. Doğrulama Verisini Dosyaya Yazma
        with open(valid_output_file, 'w', encoding='utf-8') as outfile_valid:
            for en_sent, tr_sent in valid_data:
                outfile_valid.write(f"{en_sent}\t{tr_sent}\n")
        print(f"Doğrulama verisi '{valid_output_file}' dosyasına yazıldı.")

        print("İşlem başarıyla tamamlandı.")

    except FileNotFoundError:
        print(f"Hata: '{input_file}' dosyası bulunamadı.")
    except Exception as e:
        print(f"Bir hata oluştu: {e}")


if __name__ == '__main__':
    main()

Mounted at /content/drive
'/content/drive/My Drive/transformer/en2tr.txt' dosyası okunuyor ve temizleniyor...
Toplam 473035 adet temizlenmiş cümle çifti bulundu.
Cümle çiftleri karıştırılıyor...
Veri bölündü: 425731 cümle çifti eğitim için, 47304 cümle çifti doğrulama için.
Eğitim verisi '/content/drive/My Drive/transformer/en2tr_train_1.txt' dosyasına yazıldı.
Doğrulama verisi '/content/drive/My Drive/transformer/en2tr_valid_1.txt' dosyasına yazıldı.
İşlem başarıyla tamamlandı.
